In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

import pymysql
import requests
import json
from datetime import datetime
import time
import logging

In [9]:
# 데이터 불러오기 및 필요한 함수 정의
base_src = 'Data'

# user 데이터
research_src = os.path.join(base_src, 'research_data.csv')
research = pd.read_csv(research_src,
                    sep=',',
                    encoding = 'cp949')
research = research.set_index('identifier')
research

,TIMESTAMP,job,age,gender,4. 지난 한 달 동안 책을 몇 권 읽었나요?,5. 책을 읽는 시간은 하루에 얼마나 되나요?,6. 책을 읽고 내용을 기록하고 싶은 적이 있으셨나요?,7. 책을 읽고 내용에 대해 다른 사람과 이야기하고 싶은 적이 있으셨나요?,8-1. 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요!( 평점 범위 : 0.0 ~ 5.0 ) [책 01],8-1. 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요!( 평점 범위 : 0.0 ~ 5.0 ) [평점 01],...,8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 16],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 17],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 17],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 18],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 18],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 19],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 19],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 20],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 20],9. 기프티콘 추첨을 위해 번호를 입력해주세요!
identifier,,,,,,,,,,,,,,,,,,,,,
research001,2022-09-20 17:45,취업준비행,1993,MALE,1권 ~ 2권,1시간,2,3,지적 대화를 위한 넓고 얕은 지식 1,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1058483154
research002,2022-09-20 17:45,취업준비생,1995,MALE,한 권도 읽지 않았다.,0시간,1,3,달러구트 꿈 백화점,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1064190743
research003,2022-09-20 17:45,취업준비생,1997,MALE,1권 ~ 2권,0시간,5,5,파피용,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1085500751
research004,2022-09-20 17:46,취업준비생,2000,FEMALE,한 권도 읽지 않았다.,0시간,4,3,스쿼시,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1073666880
research005,2022-09-20 17:51,취준생,1996,MALE,1권 ~ 2권,1시간,4,4,필로소피 랩,4.5,...,3.9,OpenCV 4로 배우는 컴퓨터 비전과 머신러닝,4.4,규칙없음,4.5,강경원의 back to basic,3.8,예민한 사람도 마음이 편안해지는 작은 습관,4.0,1057545028
research006,2022-09-20 17:52,취업준비생,1996,MALE,1권 ~ 2권,1시간,1,4,최재천의 공부,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023010736
research007,2022-09-20 18:01,취업준비생,1996,MALE,한 권도 읽지 않았다.,0시간,1,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082224853
research008,2022-09-20 18:01,취업준비생,1993,MALE,한 권도 읽지 않았다.,0시간,3,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
research009,2022-09-20 18:01,취업준비생,1997,MALE,한 권도 읽지 않았다.,0시간,5,5,지구 끝의 온실,3.0,...,3.5,어른은 어떻게 성장하는가,2.0,우리는 마약을 모른다,5.0,사고 싶은 컬러 팔리는 컬러,3.5,철학은 어떻게 삶의 무기가 되는가,4.0,1040761102


In [20]:
#   db 연결
def dbconnect():
    conn = pymysql.connect (host='chaekin.cvg3ycl5pbuo.ap-northeast-2.rds.amazonaws.com', user='admin', password='chrlghk1!', db='testdb', charset='utf8')
    return conn

conn = dbconnect()  #   DB 연결

In [4]:
'''
member TB에서 identifier에 해당하는 row의 id를 가져옴
'''
def find_member_id(conn, identifier):
    cur = conn.cursor()
    sql = "SELECT id FROM member WHERE identifier = %s"
    cur.execute(sql, identifier)
    result = cur.fetchall()
    conn.commit()
    return result

In [5]:
'''
member TB에 member insert
'''
def insert_member(conn, member):
    cursor = conn.cursor()

    sql = "INSERT INTO member (age, gender, identifier, is_removed, job, nickname) VALUES (%s, %s, %s, %s, %s, %s)"
    
    cursor.execute(sql, member)
    conn.commit()

In [6]:
'''
research TB에 member_id, title, score insert
'''
def insert_research(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO research (member_id, title, score) VALUES (%s, %s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()

In [17]:
'''
research TB에서 모든 record 가져오기
'''
def get_research_records(conn):
    cur = conn.cursor()
    sql = "SELECT * FROM research"
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    return result

In [78]:
'''
research TB에 isbn update
'''
def update_isbn(conn, item):
    cur = conn.cursor()
    sql = "update research set isbn = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [79]:
'''
research TB에 book_isbn_id update
'''
def update_book_isbn_id(conn, item):
    cur = conn.cursor()
    sql = "update research set book_isbn_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [66]:
'''
book_isbn TB에서 isbn에 해당하는 book_isbn_id 가져오기
'''
def get_book_isbn_id(conn, isbn):
    cur = conn.cursor()
    sql = "SELECT * FROM book_isbn where isbn = %s"
    cur.execute(sql,isbn)
    result = cur.fetchall()
    conn.commit()
    return result

In [73]:
'''
book_isbn TB에 isbn insert
'''
def insert_book_isbn(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO book_isbn (id, isbn) VALUES (%s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()

In [23]:
def is_error(res):
    # print(res)
    try:
        #   isbn으로 알라딘 api 조회시 상품이 없는 경우
        #   응답 전문으로 errorCode가 날라옴
        #   error임
        error_code = res['errorCode']
        return True
    except:
        #   errorCode가 존재하지 않으므로 사용 가능
        return False

In [54]:
'''
research TB에서 모든 record 가져오기
'''
def search_title(title):
     #   요청 api 주소
    API_HEAD = 'http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey='
    #   api key
    API_KEYS = 'ttbbeckhem961036001'
    API_BODY = '&Query='
    API_TAIL = '&QueryType=Title&MaxResults=1&start=1&SearchTarget=Book&output=js&Version=20131101'

    url = API_HEAD + API_KEYS + API_BODY + title + API_TAIL
    res = requests.get(url)
    item = res.json()['item']
    if item:
        isbn = item[0]['isbn13']
    else:
        isbn = None
    
    return isbn

In [ ]:
# 회원 정보 insert
for row in research.itertuples():
    data = (row[3], row[4], row[0], 0, row[2], row[0])
    print(data)
    insert_member(conn, data)

In [ ]:
# research TB에 member_id, title, score insert
for row in research.itertuples():
    member_id = find_member_id(conn, row[0])[0][0]
    print(member_id)
    for i in (range(9, 48, 2)):
        if pd.isna(row[i]):
            break
        print(row[i]," ",row[i+1])
        item = (member_id, row[i], row[i+1])
        insert_research(conn, item)

In [55]:
research_records = get_research_records(conn);
for row in research_records:
    isbn = search_title(row[2])
    item = (isbn, row[0])
    if item:
        update_isbn(conn, item)
    print(item, row[2])

('9791190313186', 1) 지적 대화를 위한 넓고 얕은 지식 1
('9791165341909', 2) 달러구트 꿈 백화점
('9788970127248', 3) 총균쇠
('9788932907741', 4) 파피용
('9791158392239', 5) 모던자바스크립트 딥다이브
(None, 6) 면접을 위한 CS 전공지식노트
('9791130626970', 7) 스쿼시
('9788965746898', 8) 사일런트 페이션트
('9791160946918', 9) 어린이라는 세계
('', 10) 모모
('9788937832611', 11) 헝거게임
('9788984371026', 12) 빅 픽처
('9788991239685', 13) 백설공주에게 죽음을
('9791155814116', 14) 필로소피 랩
('9791191211467', 15) 그냥 하지말라
('9791161571379', 16) 불편한 편의점
('9791185584454', 17) 타이탄의 도구들
('9788934992493', 18) 죽은 자의 집 청소
('9791195522125', 19) 언어의 온도
('9791169210003', 20) 에이트
('9788901244600', 21) 살고 싶다는 농담
('9788949170534', 22) 다다다
('9791158741396', 23) 가서 만나고 이야기하라
('9791158390600', 24) code complete 2
(None, 25) 파이썬 코딩의 기술 2판
('9788968484636', 26) 밑바닥부터 시작하는 딥러닝
('9791158391744', 27) 개발자의 글쓰기
('9791170400523', 28) 이어령의 마지막 수업
('9791162241745', 29) 밑바닥부터 시작하는 딥러닝2
('9791160507652', 30) OpenCV 4로 배우는 컴퓨터 비전과 머신러닝
('9788925599632', 31) 규칙없음
(None, 32) 강경원의 back to basic
('9791165215835', 3

('9788934996309', 266) 창가의 토토
('9788928315178', 267) 팔만대장경
(None, 268) 템테이션
('9788901232171', 269) 관찰의 기술
('9791160940121', 270) 헌법의 상상력
('9788984371118', 271) 모멘트
('9788956604992', 272) 7년의 밤
('9788959402687', 273) 촘스키, 누가 무엇으로 세상을 지배하는가
('9791197578441', 274) 지적 대화를 위한 넓고 얕은 지식
('9791170433354', 275) 숫자로 경영하라
('9788901110806', 276) 행동의 심리학
('9788970126043', 277) 위험한 관계
('9788956609959', 278) 종의 기원
(None, 279) 28
('9788934972464', 280) 싸피엔스
('9788925577630', 281) 인플레이션
('9788934977841', 282) 호모데우스
('9788937832970', 283) 브리테너
('9788960171374', 284) 세계대전z
('9791196178611', 285) 블랙홀의 사생활
('9788934996873', 286) 유발하라리의 르네상스 전쟁회고록
('9788984371026', 287) 빅픽처
('9788998658359', 288) 자기신뢰의 힘
(None, 289) 데일카네기의 인관관계론
(None, 290) 데미안
(None, 291) 미움받을 용기 (4.5)
('9788936438838', 292) 아버지의 해방일지
('9788954699914', 293) 하얼빈
('9791161571379', 294) 불편한 편의점2
(None, 295) 계속 가보겠습니다.
('9791168340541', 296) 파친코2
('9791168340510', 297) 파친코1
('9791191825145', 298) 위로의 미술관
('9788959897094', 299) 트렌드 코리아 2023
('9

In [92]:
research_records = get_research_records(conn)
index = 74
for row in research_records:
    isbn = row[4]
    book_isbn_id = get_book_isbn_id(conn, isbn)
    if len(book_isbn_id) == 0:
        item = (index, isbn)
        insert_book_isbn(conn, item)
        book_isbn_id = get_book_isbn_id(conn, isbn)
        index = index + 1
    
    item2 = (book_isbn_id[0][0], row[0])
    print(item2)
    update_book_isbn_id(conn, item2)

(5584764, 1)
(5707051, 2)
(954778, 3)
(2272408, 4)
(5773358, 5)
(1, 6)
(5572819, 7)
(5411220, 8)
(5795911, 9)
(1323591, 10)
(1006570, 11)
(1072294, 12)
(1139434, 13)
(6310172, 14)
(6310125, 15)
(2, 16)
(3646824, 17)
(5676032, 18)
(3278940, 19)
(3, 20)
(5735384, 21)
(154527, 22)
(4, 23)
(5, 24)
(6, 25)
(3405992, 26)
(5484919, 27)
(6326227, 28)
(5400448, 29)
(5391271, 30)
(5747523, 31)
(6060889, 32)
(6042837, 33)
(7, 34)
(5408151, 35)
(3754574, 36)
(3441055, 37)
(2376480, 38)
(2371850, 39)
(2373325, 40)
(8, 41)
(2373084, 42)
(2370295, 43)
(3467245, 44)
(2271872, 45)
(9, 46)
(6065686, 47)
(3356410, 48)
(6000477, 49)
(10, 50)
(3748133, 51)
(3687594, 52)
(2619670, 53)
(5385303, 54)
(5236389, 55)
(5236389, 56)
(2645419, 57)
(3877600, 58)
(5233026, 59)
(3651638, 60)
(11, 61)
(12, 62)
(5407578, 63)
(3835495, 64)
(5397261, 65)
(5357487, 66)
(2370295, 67)
(1902173, 68)
(5773358, 69)
(954778, 70)
(2376334, 71)
(2396926, 72)
(13, 73)
(3735001, 74)
(3921678, 75)
(2763903, 76)
(2792698, 77)
(3107992